In [2]:
import pandas as pd


# ## DOMA
# # Define the file path
# nodes_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
# roads_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Roads_processed.csv"
# bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"
# survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"


# Práca
# Define the file path
nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"
survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\relia\AppData\Local\Temp\ipykernel_13772\2363079150.py:20: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\relia\AppData\Local\Temp\ipykernel_13772\2363079150.py:21: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [3]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

df_bridges_all = df_bridges_all[df_bridges_all["n_2023"] != 0]

df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024'],
      dtype='object')

In [4]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[

    (df_bridges_all["Správca_úseku"] == 'SC KSK - KS') |
    (df_bridges_all["Správca_úseku"] == 'SC KSK - SNV') |
    (df_bridges_all["Správca_úseku"] == 'SC KSK - MI') |
    (df_bridges_all["Správca_úseku"] == 'SC KSK - TV') |
    (df_bridges_all["Správca_úseku"] == 'SC KSK - RV')
]

# Reindex the DataFrame
filtered_bridges.reset_index(drop=True, inplace=True)

filtered_bridges = filtered_bridges[filtered_bridges["n_2023"] != 0]

# Replace double space with single space in each item of the "Predmet_premostenia" column
filtered_bridges["Predmet_premostenia"] = filtered_bridges["Predmet_premostenia"].str.replace(
    "  ", " ")


# Display the filtered DataFrame
filtered_bridges.shape

(653, 44)

In [6]:
import geopandas as gpd
# Replace the file path with your actual file path
# file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # doma
file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson' # robota


# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(file_path)
# Display the first few rows of the GeoDataFrame

In [ ]:
import folium


def plot_filtered_bridges(df, gdf, regions):
    # Find the average coordinates for the initial map center
    avg_lat = df['Zemepisná_šírka'].mean()
    avg_long = df['Zemepisná_dĺžka'].mean()

    # Create the map with custom tileset
    m = folium.Map(location=[avg_lat, avg_long],
                   zoom_start=9, tiles="Cartodb positron")

    # Colors for the markers
    colors = ["#ff6600", "#0066FF"]

    # Bridge icon
    bridge_icon_url = "https://raw.githubusercontent.com/Medvedku/Assets/main/markers/bridge_marker_Cartodb_orng.svg"

    for index, row in df.iterrows():
        ic_size = 40
        bridge_icon = folium.CustomIcon(
            icon_image=bridge_icon_url,
            icon_size=(ic_size, ic_size),
            icon_anchor=(ic_size/2, ic_size)
        )

        popup_html = f"<table style='width:400px;'>"
        popup_html += f"<tr><th style='text-align:left;'>Názov mosta:</th><td>{row['Názov_mosta']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Trieda PK:</th><td>{row['Trieda_PK']}</td></tr>"

        popup_html += "<tr><th style='text-align:left;'>Dĺžka premostenia:</th><td>"
        if pd.notna(row['Dĺžka_premostenia']):
            popup_html += f"{float(row['Dĺžka_premostenia'])} m"
        else:
            popup_html += "Unknown"
        popup_html += "</td></tr>"

        popup_html += f"<tr><th style='text-align:left;'>Druh konštrukcie:</th><td>{row['Druh_konštrukcie']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Materiál:</th><td>{row['Material']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Rok postavenia:</th><td>{row['Rok_postavenia']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>GPS Coordinates:</th><td>({row['Zemepisná_šírka']:.4f}, {row['Zemepisná_dĺžka']:.4f})</td></tr>"
        popup_html += f"</table>"

        popup = folium.Popup(popup_html, parse_html=False)

        tooltip_html = f"<table style='width:50px;'>"
        tooltip_html += f"<tr><th colspan='2' style='text-align:left;'>Most {row['ID_mosta']}</th></tr>"
        tooltip_html += f"<tr><td style='text-align:left;'>STS:</td><td>{row['n_2023']}</td></tr>"
        tooltip_html += f"</table>"

        # Add the marker with the custom icon and popup
        folium.Marker(
            [row['Zemepisná_šírka'], row['Zemepisná_dĺžka']],
            icon=bridge_icon,
            popup=popup,
            tooltip=tooltip_html
        ).add_to(m)

    # Filter the GeoDataFrame to include only the specified regions
    region_gdf = gdf[gdf['NM4'].isin(regions)]

    # Define custom style for the regions' polygons
    def style_function(x): return {
        'fillColor': '#FF6600', 'color': '#FF6600', 'weight': 2}

    # Add the boundaries of the specified regions to the map with the custom style
    folium.GeoJson(region_gdf, name='Regions Boundaries',
                   style_function=style_function).add_to(m)

    return m


# Usage example (replace 'filtered_bridges' with your filtered DataFrame and adjust regions as needed)
regions_to_plot = ['Košický']
map = plot_filtered_bridges(filtered_bridges, gdf, regions_to_plot)
map.save('Bridges_KE.html')
map

In [7]:
import plotly.graph_objects as go

# Creating the histogram for filtered_bridges
trace_filtered = go.Histogram(
    x=filtered_bridges["n_2023"],
    name='Mosty SC KSK',
    marker_color='#e7a564 ',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=1  # Set offsetgroup to 1 for the bars to be side by side
)

# Creating the histogram for df_bridges_all
trace_all = go.Histogram(
    x=df_bridges_all["n_2023"],
    name='Všetky mosty',
    marker_color='#46a7a2',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=2  # Set offsetgroup to 2 for the bars to be side by side
)

# Creating the figure and adding the histograms
fig = go.Figure(data=[trace_filtered, trace_all])

# Update the layout with Montserrat font
fig.update_layout(
    title_text='Relatívne zastúpenie STS mostov',
    xaxis_title_text='stavebno-technický stav',
    yaxis_title_text='zastúpenie (%)',
    barmode='group',  # Set barmode to "group" for bars to be side by side
    font=dict(
        family="Montserrat, sans-serif",  # Set font family to Montserrat
        size=14
    ),
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=1.10,  # Position the legend slightly above the plot
        xanchor="left",  # Anchor legend to the left
        x=.615  # Align legend to the left
    ),
    plot_bgcolor='#EEEEEE',
    margin=dict(l=0, r=20, t=50, b=50),  # Adjust margins for tight layout
    bargap=0.2,
    width=800,
    height=400
)

# Adjusting the opacity for better visualization
fig.update_traces(opacity=0.75)

fig.show()

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

# Initialize an empty DataFrame for relative condition counts across years
relative_condition_counts = pd.DataFrame()

for year in range(2012, 2025):
    column_name = f"n_{year}"
    # Exclude bridges not assessed in the year (condition = 0)
    year_data = filtered_bridges[filtered_bridges[column_name] > 0]  # NR
    # year_data = df_bridges_all[df_bridges_all[column_name] > 0] # Všetky mosty
    # Count occurrences of each condition for the year
    counts = year_data[column_name].value_counts().rename_axis(
        'Condition').reset_index(name=column_name)
    # Calculate relative counts (percentage) for the year
    total = counts[column_name].sum()
    counts[column_name] = (counts[column_name] / total) * 100

    if relative_condition_counts.empty:
        relative_condition_counts = counts
    else:
        relative_condition_counts = pd.merge(
            relative_condition_counts, counts, on='Condition', how='outer')

# Fill NaN values with 0 for years when a condition was not present
relative_condition_counts.fillna(0, inplace=True)
# Sort the DataFrame by 'Condition' for a consistent order
relative_condition_counts.sort_values('Condition', inplace=True)

In [31]:
import plotly.graph_objects as go

# Define the color palette
color_palette = ['#46a7a2', '#e7a564',
                 '#8bae7c', '#353a29', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Transpose the DataFrame for plotting: years as columns, conditions as rows
data_for_plotting = relative_condition_counts.set_index(
    'Condition').T.reset_index()
data_for_plotting.rename(columns={'index': 'Year'}, inplace=True)

# Update year labels to include 2024
years = ['{}'.format(year) for year in range(2012, 2025)]
data_for_plotting['Year'] = years

# Create a figure
fig = go.Figure()

# Add a trace for each condition in reverse order
for i, condition in reversed(list(enumerate(relative_condition_counts['Condition'].unique()))):
    fig.add_trace(go.Scatter(
        x=data_for_plotting['Year'],
        y=data_for_plotting[condition],
        mode='lines',
        stackgroup='one',  # This option stacks the areas
        name=str(condition),  # Label for the legend
        line=dict(color=color_palette[i % len(color_palette)])
    ))

# Update the layout to include 2024
fig.update_layout(
    title='Vývoj STS mostov SC KSK v čase',
    yaxis_title='percentuálne zastúpenie',
    yaxis=dict(ticksuffix="%", range=[0, 100]),
    font=dict(family="Montserrat, sans-serif", size=14),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.120,
        xanchor="right",
        x=1
    ),
    xaxis=dict(
        tickvals=[str(year) for year in range(2012, 2025)],
        ticktext=['2012', '2013', '2014', '2015', '2016', '2017',
                  '2018', '2019', '2020', '2021', '2022', '2023', '2024']
    ),
    plot_bgcolor='#EEEEEE',
    margin=dict(l=0, r=20, t=50, b=25),
    width=800,
    height=400
)

# Show the figure
fig.show()


In [10]:
filtered_bridges["Material"].value_counts(), len(
    list(filtered_bridges["Material"].value_counts()))

(Material
 monolitický železobetón          266
 prefabrikovaný železobetón       184
 prefabrikovaný predpätý betón    144
 kameň                             25
 oceľ -  plnostenná                13
 monolitický prostý betón          12
 monolitický predpätý betón         7
 spriahnutá oceľ-betón              1
 spriahnutá betón-betón             1
 Name: count, dtype: int64,
 9)

In [11]:
import plotly.graph_objects as go

# Define the color palette
color_palette = ['#e7a564', '#46a7a2', '#c6d9df', '#b17431',
                 '#8bae7c', '#353a29', '#f1aab2', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Get the material counts
material_counts = filtered_bridges["Material"].value_counts()

# Create a bar plot
fig = go.Figure()

# Add traces for each material, include the count as text, and set textposition to 'auto' for automatic placement
for material, count, color in zip(material_counts.index, material_counts.values, color_palette):
    fig.add_trace(go.Bar(
        x=[material],  # Set x to material for vertical bars
        y=[count],
        name=material,
        marker_color=color,
        text=[count],  # Add the count as text for each bar
        textposition='outside'  # Position the text outside of bars
    ))

# Update the layout with additional configurations
fig.update_layout(
    title="Konštrukčné materiály mostov v správe SC KSK",
    yaxis_title="počet",
    xaxis=dict(
        showticklabels=False
    ),
    plot_bgcolor='#EEEEEE',  # Set plot background color
    font=dict(family="Montserrat, sans-serif",
              size=14),  # Set font family and size
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=0.975,  # Position the legend slightly above the plot
        xanchor="right",  # Anchor legend to the right
        x=0.985  # Align legend to the right
    ),
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    bargap=0.3,  # Increase the gap between bars for more space
    width=800,
    height=400
)

# Add a custom x-axis label using annotations
fig.update_layout(
    annotations=[
        dict(
            x=0.5,  # x-coordinate position (0.5 for center)
            y=-0.075,  # y-coordinate position (-0.15 places it below the bars)
            xref='paper',  # Use 'paper' for positioning relative to the plot area
            yref='paper',  # Use 'paper' for positioning relative to the plot area
            showarrow=False,  # Do not show an arrow
            text=" ",  # The text you want as the x-axis label
            # Match the font size with the rest of the plot
            font=dict(family="Montserrat, sans-serif",
                      size=15),
        )
    ]
)

# Show the updated plot
fig.show()

In [12]:
import plotly.graph_objects as go

# Get the material counts
material_counts = filtered_bridges["Material"].value_counts()

# Calculate the total count of bridges
total_count = material_counts.sum()

# Calculate the relative frequency of each material
relative_frequency = material_counts / total_count * 100

# Define the color palette
color_palette = ['#e7a564', '#46a7a2', '#c6d9df', '#b17431',
                 '#8bae7c', '#353a29', '#f1aab2', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Create a bar plot
fig = go.Figure()

# Add traces for each material, include the relative frequency as text, and set textposition to 'auto' for automatic placement
for material, frequency, color in zip(relative_frequency.index, relative_frequency.values, color_palette):
    fig.add_trace(go.Bar(
        x=[material],  # Set x to material for vertical bars
        y=[frequency],
        name=material,
        marker_color=color,
        # Add the relative frequency as text for each bar
        text=[f"{frequency:.2f}%"],
        textposition='outside'  # Position the text outside of bars
    ))

# Update the layout with additional configurations
fig.update_layout(
    title="Konštrukčné materiály mostov v správe RSÚC NR",
    yaxis_title="percentuálne zastúpenie",
    xaxis=dict(
        showticklabels=False
    ),
    plot_bgcolor='#EEEEEE',  # Set plot background color
    font=dict(family="Montserrat, sans-serif",
              size=14),  # Set font family and size
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=0.975,  # Position the legend slightly above the plot
        xanchor="right",  # Anchor legend to the right
        x=0.985  # Align legend to the right
    ),
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    bargap=0.3,  # Increase the gap between bars for more space
    width=800,
    height=400
)

# Add a custom x-axis label using annotations
fig.update_layout(
    annotations=[
        dict(
            x=0.5,  # x-coordinate position (0.5 for center)
            y=-0.075,  # y-coordinate position (-0.15 places it below the bars)
            xref='paper',  # Use 'paper' for positioning relative to the plot area
            yref='paper',  # Use 'paper' for positioning relative to the plot area
            showarrow=False,  # Do not show an arrow
            text="materiál",  # The text you want as the x-axis label
            # Match the font size with the rest of the plot
            font=dict(family="Montserrat, sans-serif",
                      size=15),
        )
    ]
)

# Show the updated plot
fig.show()

In [13]:
filtered_bridges["Dĺžka_premostenia"].describe()

count    653.000000
mean      13.292807
std       22.384665
min        2.050000
25%        4.000000
50%        6.000000
75%       12.100000
max      271.230000
Name: Dĺžka_premostenia, dtype: float64

In [14]:
import pandas as pd
import plotly.graph_objects as go

# Assuming filtered_bridges and color_palette are defined as before

# Use qcut to divide the data into bins with equal number of bridges in each
filtered_bridges['Quantile_Bin'] = pd.qcut(
    filtered_bridges['Dĺžka_premostenia'], q=5)

# Count the number of bridges in each bin
bin_counts = filtered_bridges['Quantile_Bin'].value_counts().sort_index()

# Create the histogram
fig = go.Figure()

# Adding bin counts as text, positioned at the base of each bar
fig.add_trace(go.Bar(
    # Convert bin interval to string for plotting
    x=bin_counts.index.astype(str),
    y=bin_counts.values,
    text=bin_counts.values,  # Set text to be the bin counts
    textposition='inside',  # Position the text inside the bars
    # Anchor text to the start (bottom for vertical bars)
    insidetextanchor='start',
    textfont=dict(  # Customize the font of the text
        color='white',  # Set text color to white
        size=14  # Match the font size with the rest of the plot
    ),
    marker_color=color_palette[:len(bin_counts)]  # Apply the color palette
))

# Update the layout
fig.update_layout(
    title="Kategorizácia mostov v správe RSÚC NR podľa ich dĺžky",
    xaxis_title="kategórie dĺžok",
    yaxis_title="počet mostov",
    plot_bgcolor='#EEEEEE',
    font=dict(family="Montserrat, sans-serif", size=14),
    bargap=0.2,
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    width=800,  # Set figure width
    height=400  # Set figure height

)

# Show the plot
fig.show()

In [15]:
import pandas as pd
import plotly.graph_objects as go

# Assuming filtered_bridges and color_palette are defined as before

# Use qcut to divide the data into bins with equal number of bridges in each
filtered_bridges['Quantile_Bin'] = pd.qcut(
    filtered_bridges['Dĺžka_premostenia'], q=5)

# Count the number of bridges in each bin
bin_counts = filtered_bridges['Quantile_Bin'].value_counts().sort_index()

# # Define custom x-axis labels based on the bin edges
custom_xticks = ["do 3.5 m", "od 3.5 m do 5.0 m",
                 "od 5.0 m do 7.8 m", "od 7.8 m do 14.72 m", "od 14.72 m"]

# Create the histogram
fig = go.Figure()

# Add bar trace with custom x-axis labels
fig.add_trace(go.Bar(
    x=custom_xticks,  # Use custom labels as x values
    y=bin_counts.values,
    text=bin_counts.values,  # Set text to be the bin counts
    textposition='inside',  # Position the text inside the bars
    insidetextanchor='start',
    textfont=dict(
        color='white',  # Set text color to white
        size=14  # Match the font size with the rest of the plot
    ),
    marker_color=color_palette[:len(bin_counts)]  # Apply the color palette
))

# Update the layout
fig.update_layout(
    title="Kategorizácia mostov v správe SC KSK podľa ich dĺžky",
    # xaxis_title="kategórie dĺžok",
    yaxis_title="počet mostov",
    plot_bgcolor='#EEEEEE',
    font=dict(family="Montserrat, sans-serif", size=14),
    bargap=0.2,
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    width=850,  # Set figure width
    height=400  # Set figure height
)

# Show the plot
fig.show()

In [16]:
filtered_bridges["Rok_postavenia"].describe()

count     653.000000
mean     1958.592649
std        20.964754
min      1875.000000
25%      1948.000000
50%      1960.000000
75%      1967.000000
max      2022.000000
Name: Rok_postavenia, dtype: float64

In [17]:
import pandas as pd
import plotly.graph_objects as go

# Assuming filtered_bridges and color_palette are defined as before

# Use qcut to divide the data into bins with equal number of bridges in each
filtered_bridges['Quantile_Bin_year'] = pd.qcut(
    filtered_bridges['Rok_postavenia'], q=3)

# Count the number of bridges in each bin
bin_counts = filtered_bridges['Quantile_Bin_year'].value_counts().sort_index()

# Create the histogram
fig = go.Figure()

# Adding bin counts as text, positioned at the base of each bar
fig.add_trace(go.Bar(
    # Convert bin interval to string for plotting
    x=bin_counts.index.astype(str),
    y=bin_counts.values,
    text=bin_counts.values,  # Set text to be the bin counts
    textposition='inside',  # Position the text inside the bars
    # Anchor text to the start (bottom for vertical bars)
    insidetextanchor='start',
    textfont=dict(  # Customize the font of the text
        color='white',  # Set text color to white
        size=14  # Match the font size with the rest of the plot
    ),
    marker_color=color_palette[:len(bin_counts)]  # Apply the color palette
))

# Update the layout
fig.update_layout(
    title="Kategorizácia mostov v správe RSÚC NR podľa roku výstavby",
    xaxis_title="kategórie veku",
    yaxis_title="počet mostov",
    plot_bgcolor='#f0f0f0',
    font=dict(family="Montserrat, sans-serif", size=14),
    bargap=0.2,
    margin=dict(l=0, r=20, t=50, b=30),
    width=800,
    height=400
)

# Show the plot
fig.show()

In [18]:
import pandas as pd
import plotly.graph_objects as go

# Assuming filtered_bridges, bin_counts, and color_palette are defined as in the previous snippet

# Define custom x-axis tick labels
custom_tick_labels = ["do roku 1952", "1952 až 1964", "od roku 1964"]

# Create the histogram with updated x-axis ticks
fig = go.Figure()
fig.add_trace(go.Bar(
    x=custom_tick_labels,  # Use custom labels as x values
    y=bin_counts.values,
    text=bin_counts.values,  # Set text to be the bin counts
    textposition='inside',  # Position the text inside the bars
    # Anchor text to the start (bottom for vertical bars)
    insidetextanchor='start',
    textfont=dict(  # Customize the font of the text
        color='white',  # Set text color to white
        size=14  # Match the font size with the rest of the plot
    ),
    marker_color=color_palette[:len(bin_counts)]  # Apply the color palette
))

# Update the layout
fig.update_layout(
    title="Kategorizácia mostov v správe SC KSK podľa roku výstavby",
    # xaxis_title="kategórie veku",
    yaxis_title="počet mostov",
    plot_bgcolor='#EEEEEE',
    font=dict(family="Montserrat, sans-serif", size=14),
    bargap=0.2,
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    width=850,  # Set figure width
    height=400  # Set figure height
)

# Show the plot
fig.show()

In [19]:
import plotly.graph_objects as go

# Define the color palette
color_palette = ['#e7a564', '#46a7a2', '#c6d9df', '#b17431',
                 '#8bae7c', '#353a29', '#f1aab2', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Get the obstacle counts
obstacle_counts = filtered_bridges["Predmet_premostenia"].value_counts()

# Create a bar plot
fig = go.Figure()

# Add traces for each obstacle type, include the count as text, and set textposition to 'auto' for automatic placement
for obstacle, count, color in zip(obstacle_counts.index, obstacle_counts.values, color_palette):
    fig.add_trace(go.Bar(
        x=[obstacle],  # Set x to obstacle for vertical bars
        y=[count],
        name=obstacle,
        marker_color=color,
        text=[count],  # Add the count as text for each bar
        textposition='outside'  # Position the text outside of bars
    ))

# Update the layout with additional configurations
fig.update_layout(
    title="Predmety premostenia mostov v správe RSÚC NR",
    yaxis_title="počet",
    xaxis=dict(
        showticklabels=False
    ),
    plot_bgcolor='#EEEEEE',  # Set plot background color
    font=dict(family="Montserrat, sans-serif",
              size=14),  # Set font family and size
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=0.975,  # Position the legend slightly above the plot
        xanchor="right",  # Anchor legend to the right
        x=0.985  # Align legend to the right
    ),
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    bargap=0.3,  # Increase the gap between bars for more space
    width=800,
    height=400
)

# Add a custom x-axis label using annotations
fig.update_layout(
    annotations=[
        dict(
            x=0.5,  # x-coordinate position (0.5 for center)
            # y-coordinate position (-0.075 places it below the bars)
            y=-0.075,
            xref='paper',  # Use 'paper' for positioning relative to the plot area
            yref='paper',  # Use 'paper' for positioning relative to the plot area
            showarrow=False,  # Do not show an arrow
            text="predmet premostenia",  # The text you want as the x-axis label
            # Match the font size with the rest of the plot
            font=dict(family="Montserrat, sans-serif",
                      size=14),
        )
    ]
)

# Show the updated plot
fig.show()

In [20]:
import plotly.graph_objects as go

# Define the color palette
color_palette = ['#e7a564', '#46a7a2', '#c6d9df', '#b17431',
                 '#8bae7c', '#353a29', '#f1aab2', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Get the obstacle counts
obstacle_counts = filtered_bridges["Predmet_premostenia"].value_counts()

# Create a bar plot
fig = go.Figure()

# Add traces for each obstacle type, include the count as text, and set textposition to 'auto' for automatic placement
for obstacle, count, color in zip(obstacle_counts.index, obstacle_counts.values, color_palette):
    fig.add_trace(go.Bar(
        x=[obstacle],  # Set x to obstacle for vertical bars
        y=[count],
        name=obstacle,
        marker_color=color,
        text=[count],  # Add the count as text for each bar
        textposition='outside'  # Position the text outside of bars
    ))

# Update the layout with additional configurations
fig.update_layout(
    title="Predmety premostenia mostov v správe RSÚC NR",
    yaxis_title="počet",
    xaxis=dict(
        showticklabels=False
    ),
    plot_bgcolor='#EEEEEE',  # Set plot background color
    font=dict(family="Montserrat, sans-serif",
              size=14),  # Set font family and size
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=0.975,  # Position the legend slightly above the plot
        xanchor="right",  # Anchor legend to the right
        x=0.985,  # Align legend to the right
        # Set legend background color with some transparency
        bgcolor='rgba(255, 255, 255, 0.7)'
    ),
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    bargap=0.3,  # Increase the gap between bars for more space
    width=800,
    height=400
)

# Add a custom x-axis label using annotations
fig.update_layout(
    annotations=[
        dict(
            x=0.5,  # x-coordinate position (0.5 for center)
            # y-coordinate position (-0.075 places it below the bars)
            y=-0.075,
            xref='paper',  # Use 'paper' for positioning relative to the plot area
            yref='paper',  # Use 'paper' for positioning relative to the plot area
            showarrow=False,  # Do not show an arrow
            text="predmet premostenia",  # The text you want as the x-axis label
            # Match the font size with the rest of the plot
            font=dict(family="Montserrat, sans-serif",
                      size=14),
        )
    ]
)

# Show the updated plot
fig.show()

In [21]:
import pandas as pd
from collections import Counter

# Splitting the strings and flattening the list
all_items = []
for item in filtered_bridges["Predmet_premostenia"]:
    split_items = [x.strip() for x in item.split(',')]
    all_items.extend(split_items)

# Counting occurrences of each unique item
item_counts = Counter(all_items)

# Output the counts
len(item_counts)

9

In [22]:
filtered_bridges["Predmet_premostenia"].value_counts()

Predmet_premostenia
Vodný tok               528
Kanál                    88
Železnica                13
Záplavové územie          7
Vodná nádrž               5
Cestná komunikácia        3
Nezistený                 3
Vodný tok, Vodný tok      2
Účelová komunikácia       1
Miestna komunikácia       1
Vodný tok, Železnica      1
Vodný tok, Kanál          1
Name: count, dtype: int64

In [23]:
filtered_bridges["Správca_úseku"].value_counts()

Správca_úseku
SC KSK - KS     176
SC KSK - SNV    164
SC KSK - MI     144
SC KSK - TV      91
SC KSK - RV      78
Name: count, dtype: int64

In [24]:
import plotly.graph_objects as go

# Get the counts of bridge sections for each administrator
administrator_counts = filtered_bridges["Správca_úseku"].value_counts()

color_palette = ['#e7a564', '#46a7a2', '#2d505d',
                 '#8bae7c', '#c25469']

# Create a bar plot
fig = go.Figure()

# Add traces for each administrator, include the count as text, and set textposition to 'auto' for automatic placement
for admin, count, color in zip(administrator_counts.index, administrator_counts.values, color_palette):
    fig.add_trace(go.Bar(
        x=[admin],  # Set x to administrator for vertical bars
        y=[count],
        name=admin,
        marker_color=color,
        text=[count],  # Add the count as text for each bar
        textposition='auto',  # Position the text automatically
        showlegend=False  # Do not show this trace in the legend
    ))

# Update the layout with additional configurations
fig.update_layout(
    title="Mosty SC KSK v správe jednotlivých oblastných prevádzok",
    yaxis_title="počet",
    xaxis=dict(
        # title="oblastné prevádzky Správy ciest Košického samosprávneho kraja",
        title=" ",
        tickvals=list(range(len(administrator_counts))),
        ticktext=["Košice",
                  "Spišská Nová Ves",
                  "Michalovce",
                  "Trebišov",
                  "Rožňava"
        
        ]
    ),
    plot_bgcolor='#EEEEEE',  # Set plot background color
    font=dict(family="Montserrat, sans-serif",
              size=14),  # Set font family and size
    margin=dict(l=0, r=20, t=50, b=50),  # Adjust margins for tight layout
    width=800,  # Set figure width
    height=300  # Set figure height
)

# Show the plot
fig.show()

In [25]:
import plotly.graph_objects as go

# Define the color palette
color_palette = ['#e7a564', '#46a7a2', '#c6d9df', '#b17431',
                 '#8bae7c', '#353a29', '#f1aab2', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Create a bar plot
fig = go.Figure()

# Add traces for each obstacle type, include the count as text, and set textposition to 'auto' for automatic placement
for obstacle, count, color in zip(item_counts.keys(), item_counts.values(), color_palette):
    fig.add_trace(go.Bar(
        x=[obstacle],  # Set x to obstacle for vertical bars
        y=[count],
        name=obstacle,
        marker_color=color,
        text=[count],  # Add the count as text for each bar
        textposition='outside'  # Position the text outside of bars
    ))

# Update the layout with additional configurations
fig.update_layout(
    title="Predmety premostenia mostov v správe SC KSK",
    yaxis_title="počet",
    xaxis=dict(
        showticklabels=True  # Show x-axis tick labels
    ),
    plot_bgcolor='#EEEEEE',  # Set plot background color
    font=dict(family="Montserrat, sans-serif",
              size=14),  # Set font family and size
    legend=dict(
        orientation="v",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=0.975,  # Position the legend slightly above the plot
        xanchor="right",  # Anchor legend to the right
        x=0.985  # Align legend to the right
    ),
    margin=dict(l=0, r=20, t=50, b=30),  # Adjust margins for tight layout
    bargap=0.3,
    width=850,
    height=450
)

# Add a custom x-axis label using annotations
fig.update_layout(
    annotations=[
        dict(
            x=0.5,  # x-coordinate position (0.5 for center)
            # y-coordinate position (-0.075 places it below the bars)
            y=-0.075,
            xref='paper',  # Use 'paper' for positioning relative to the plot area
            yref='paper',  # Use 'paper' for positioning relative to the plot area
            showarrow=False,  # Do not show an arrow
            text=" ",  # The text you want as the x-axis label
            # Match the font size with the rest of the plot
            font=dict(family="Montserrat, sans-serif",
                      size=14),
        )
    ]
)
fig.update_layout(
    xaxis=dict(showticklabels=False))

# Show the updated plot
fig.show()

In [26]:
# Filter out rows where "n_2012" is not equal to 0
non_zero_values = filtered_bridges[filtered_bridges["n_2024"] != 0]

# Calculate the mean value of the filtered "n_2012" column
mean_value_excluding_zero = non_zero_values["n_2024"].mean()

# Print the result
print("Mean value of n_2024 excluding 0 values:", mean_value_excluding_zero)

Mean value of n_2024 excluding 0 values: 4.7687595712098005


In [27]:
# Filter out rows where "n_2012" is not equal to 0
non_zero_values = filtered_bridges[filtered_bridges["n_2012"] != 0]

# Calculate the mean value of the filtered "n_2012" column
mean_value_excluding_zero = non_zero_values["n_2012"].mean()

# Print the result
print("Mean value of n_2012 excluding 0 values:", mean_value_excluding_zero)

Mean value of n_2012 excluding 0 values: 3.2569659442724457


In [28]:
filtered_bridges[["ID_mosta", "Zemepisná_šírka", "Zemepisná_dĺžka"]]

,ID_mosta,Zemepisná_šírka,Zemepisná_dĺžka
0,M4786,48.635671,20.334794
1,M663,48.635961,20.341739
2,M424,48.660173,20.365448
3,M2226,48.660301,20.365838
4,M4574,48.667657,20.405645
...,...,...,...
648,M9492,48.612000,21.241350
649,M9493,48.616650,21.240870
650,M9659,48.646500,20.370550
651,M9691,48.740870,21.345630


In [29]:
filtered_bridges[["ID_mosta", 'Trieda_PK', "Číslo_PK", "Zemepisná_šírka", "Zemepisná_dĺžka"]].to_csv(
    "filtered_bridges.csv", sep=";", index=False)

In [30]:
filtered_bridges.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024', 'Quantile_Bin',
       'Quantile_Bin_year'],
      dtype='object')